In [1]:
import json
import urllib
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf

In [6]:
cf.go_offline(connected=True)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
def get_data(service, params):
    params['lang'] = 'kr'
    params['auth'] = '7774090942ede970746a7cd9b2e10577'
    tmp = []
    for k, v in params.items():
        tmp.append(k + "=" + v)
    query = '&'.join(tmp)
    url = 'http://fisis.fss.or.kr/openapi/{}.json?'.format(service) + query
    res = urllib.request.urlopen(url)
    data = json.loads(res.read())
    return data

In [261]:
# 손해율
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI136',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

loss_ratio = []
for company in company_list.keys():
    params['financeCd'] = company
    loss_ratio.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
loss_ratio = pd.concat(loss_ratio).query('account_cd == "D"').reset_index(drop=True)
loss_ratio['finance_nm'] = loss_ratio['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
loss_ratio.columns = list(map(lambda x: columns.get(x, x), loss_ratio.columns))
loss_ratio['base_month'] = loss_ratio['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
loss_ratio['비율'] = loss_ratio['비율'].astype(float)
loss_ratio = loss_ratio.pivot_table(index='base_month', columns='finance_nm', values='비율', aggfunc=np.sum)
loss_ratio.iplot(kind='line', mode='lines+markers')

In [217]:
# 합산비율
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI136',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

comb_ratio = []
for company in company_list.keys():
    params['financeCd'] = company
    comb_ratio.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
comb_ratio = pd.concat(comb_ratio).query('account_cd == "L"').reset_index(drop=True)
comb_ratio['finance_nm'] = comb_ratio['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
comb_ratio.columns = list(map(lambda x: columns.get(x, x), comb_ratio.columns))
comb_ratio['base_month'] = comb_ratio['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
comb_ratio['비율'] = comb_ratio['비율'].astype(float)
comb_ratio = comb_ratio.pivot_table(index='base_month', columns='finance_nm', values='비율', aggfunc=np.sum)
comb_ratio.iplot(kind='line', mode='lines+markers')

In [209]:
# 운용자산
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI003',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

oper_asset = []
for company in company_list.keys():
    params['financeCd'] = company
    oper_asset.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
oper_asset = pd.concat(oper_asset).query('account_cd == "A1"').reset_index(drop=True)
oper_asset['finance_nm'] = oper_asset['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
oper_asset.columns = list(map(lambda x: columns.get(x, x), oper_asset.columns))
oper_asset['base_month'] = oper_asset['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
oper_asset['금액'] = oper_asset['금액'].astype(float)
oper_asset = oper_asset.pivot_table(index='base_month', columns='finance_nm', values='금액', aggfunc=np.sum)
oper_asset = np.round(oper_asset/1e12,2)
oper_asset.iplot(kind='line', mode='lines+markers')

In [226]:
# 책임준비금
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI123',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

reserve = []
for company in company_list.keys():
    params['financeCd'] = company
    reserve.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
reserve = pd.concat(reserve).query('account_cd == "A"').reset_index(drop=True)
reserve['finance_nm'] = reserve['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
reserve.columns = list(map(lambda x: columns.get(x, x), reserve.columns))
reserve['base_month'] = reserve['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
reserve['금액'] = reserve['금액'].astype(float)
reserve = reserve.pivot_table(index='base_month', columns='finance_nm', values='금액', aggfunc=np.sum)
reserve = np.round(reserve/1e12,2)
reserve.iplot(kind='line', mode='lines+markers')

In [235]:
# 이익잉여금
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI004',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

ret_earnings = []
for company in company_list.keys():
    params['financeCd'] = company
    ret_earnings.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
ret_earnings = pd.concat(ret_earnings).query('account_cd == "A24"').reset_index(drop=True)
ret_earnings['finance_nm'] = ret_earnings['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
ret_earnings.columns = list(map(lambda x: columns.get(x, x), ret_earnings.columns))
ret_earnings['base_month'] = ret_earnings['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
ret_earnings['금액'] = ret_earnings['금액'].astype(float)
ret_earnings = ret_earnings.pivot_table(index='base_month', columns='finance_nm', values='금액', aggfunc=np.sum)
ret_earnings = np.round(ret_earnings/1e12,2)
ret_earnings.iplot(kind='line', mode='lines+markers')

In [248]:
# 당기순이익
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI007',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

total_profit = []
for company in company_list.keys():
    params['financeCd'] = company
    total_profit.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
total_profit = pd.concat(total_profit).query('account_cd == "U"').reset_index(drop=True)
total_profit['finance_nm'] = total_profit['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
total_profit.columns = list(map(lambda x: columns.get(x, x), total_profit.columns))
total_profit['base_month'] = total_profit['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
total_profit['당분기'] = total_profit['당분기'].astype(float)
total_profit = total_profit.pivot_table(index='base_month', columns='finance_nm', values='당분기', aggfunc=np.sum)
total_profit = np.round(total_profit/1e8,0)
total_profit.iplot(kind='line', mode='lines+markers')

In [249]:
# 기타포괄손익
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI007',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

total_profit = []
for company in company_list.keys():
    params['financeCd'] = company
    total_profit.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
total_profit = pd.concat(total_profit).query('account_cd == "Z"').reset_index(drop=True)
total_profit['finance_nm'] = total_profit['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
total_profit.columns = list(map(lambda x: columns.get(x, x), total_profit.columns))
total_profit['base_month'] = total_profit['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
total_profit['당분기'] = total_profit['당분기'].astype(float)
total_profit = total_profit.pivot_table(index='base_month', columns='finance_nm', values='당분기', aggfunc=np.sum)
total_profit = np.round(total_profit/1e8,0)
total_profit.iplot(kind='line', mode='lines+markers')

In [236]:
# 기타포괄손익누계액
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI004',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

ret_earnings = []
for company in company_list.keys():
    params['financeCd'] = company
    ret_earnings.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
ret_earnings = pd.concat(ret_earnings).query('account_cd == "A26"').reset_index(drop=True)
ret_earnings['finance_nm'] = ret_earnings['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
ret_earnings.columns = list(map(lambda x: columns.get(x, x), ret_earnings.columns))
ret_earnings['base_month'] = ret_earnings['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
ret_earnings['금액'] = ret_earnings['금액'].astype(float)
ret_earnings = ret_earnings.pivot_table(index='base_month', columns='finance_nm', values='금액', aggfunc=np.sum)
ret_earnings = np.round(ret_earnings/1e12,2)
ret_earnings.iplot(kind='line', mode='lines+markers')

In [175]:
# 사업실적표
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI020',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

biz_perf = []
for company in company_list.keys():
    params['financeCd'] = company
    biz_perf.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
biz_perf = pd.concat(biz_perf).reset_index(drop=True)
biz_perf['finance_nm'] = biz_perf['finance_cd'].apply(lambda x: company_list.get(x))
columns = dict(map(lambda x: (x['column_id'], x['column_nm']), get_data(service_code, params)['result']['description']))
biz_perf.columns = list(map(lambda x: columns.get(x, x), biz_perf.columns))
biz_perf['base_month'] = biz_perf['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))

In [176]:
# 원수보험료
ori_prem = biz_perf.query('account_cd == "A3"')
ori_prem['합계'] = ori_prem['합계'].astype(float)
ori_prem = ori_prem.pivot_table(index='base_month', columns='finance_nm', values='합계', aggfunc=np.sum)
ori_prem_incr = ori_prem.copy()
for i in range(len(ori_prem_incr)):
    ori_prem_incr.iloc[i] = ori_prem.iloc[i] if ori_prem_incr.iloc[i].name.month == 3 else ori_prem.iloc[i]-ori_prem.iloc[i-1]
ori_prem_incr = np.round(ori_prem_incr/1e12, 2)
ori_prem_incr.iplot(kind='line', mode='lines+markers')

In [247]:
# 법인세차감전순이익
ebit = biz_perf.query('account_cd == "K"')
ebit['합계'] = ebit['합계'].astype(float)
ebit = ebit.pivot_table(index='base_month', columns='finance_nm', values='합계', aggfunc=np.sum)
ebit_incr = ebit.copy()
for i in range(len(ebit_incr)):
    ebit_incr.iloc[i] = ebit.iloc[i] if ebit_incr.iloc[i].name.month == 3 else ebit.iloc[i]-ebit.iloc[i-1]
ebit_incr = np.round(ebit_incr/1e8, 0)
ebit_incr.iplot(kind='line', mode='lines+markers')

In [271]:
# RBC비율
company_list = {'0010633': '삼성', '0010636': 'DB', '0010634': '현대', '0010635': 'KB', '0010626': '메리츠'}

service_code = 'statisticsInfoSearch'
params = {
    'listNo': 'SI139',
    'term': 'Q',
    'startBaseMm': '201501',
    'endBaseMm': '201912',
}

rbc_ratio = []
for company in company_list.keys():
    params['financeCd'] = company
    rbc_ratio.append(pd.DataFrame(get_data(service_code, params)['result']['list']))
rbc_ratio = pd.concat(rbc_ratio).reset_index(drop=True)
rbc_ratio['a'] = rbc_ratio['a'].astype(float)
rbc_ratio['base_month'] = rbc_ratio['base_month'].apply(lambda x: datetime(int(x[:4]), int(x[-2:]), 1) + relativedelta(months=1, days=-1))
rbc_ratio['finance_nm'] = rbc_ratio['finance_cd'].apply(lambda x: company_list.get(x))
rbc_ratio = rbc_ratio.pivot_table(index='base_month', columns='finance_nm', values='a', aggfunc=np.sum)
rbc_ratio.iplot(kind='line', mode='lines+markers')

In [292]:
# fig = rbc_ratio.iplot(kind='line', mode='lines+markers', asFigure=True)
# fig.write_json('test.json')

# import plotly

# fig = {'data': [{'x': [1,2,3],
#                   'y': [2,5,3],
#                   'type': 'bar'}],
#       'layout': {'width': 800,
#                  'height': 550}}

# aPlot = plotly.offline.plot(fig, 
#                             config={"displayModeBar": True}, 
#                             show_link=False, 
#                             include_plotlyjs=False, 
#                             output_type='div')

# html_string = '''
# <html>
#     <head>
#       <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
#       <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
#       <style>body{ margin:0 100; background:whitesmoke; }</style>
#     </head>
#     <body>
#       <h1>Monthly Report</h1>
#       ''' + aPlot + '''
#     </body>
# </html>'''

# with open("nick.html", 'w') as f:
#     f.write(html_string)

In [9]:
with pd.ExcelWriter('fss_result.xlsx', 'xlsxwriter') as writer:
    rbc_ratio.to_excel(writer, 'RBC비율', index=False)